# Notebook to create invididual pages for web pages


In [ ]:
%config InlineBackend.figure_formats = ['svg']

import datetime
import os
import random
import pandas as pd
import matplotlib.pyplot as plt

import coronavirus as corona

pd.set_option('display.float_format', '{:.2f}'.format)  #  Disable pandas scientific notation
# force download of new data
# clear_cache()



# fetch world data
d, c = corona.fetch_deaths(), corona.fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

def save_frontpage_figure(country):
    fig, ax =  plt.subplots(figsize=(12, 2.5))
    cases, deaths = corona.get_country_data(country)
    corona.plot_daily_change(ax, cases, 'C1')
    filename = 'fig-homepage-example-plot'
    outdir = 'pelican/content/pages'
    outpath = os.path.join(outdir, filename)
    
    # create fake point for extra text in legend
    xvalue = (ax.get_xlim()[0] + ax.get_xlim()[1])*0.5
    yvalue = (ax.get_ylim()[0] + ax.get_ylim()[1])*0.5
    date_string = datetime.datetime.now().strftime("%d/%m/%Y")  # %H:%M") + " CET"
    extra = ax.plot([xvalue], [yvalue], 'o', alpha=0.001, 
                    label=f"[last updated: {date_string}]")
    
    # change order of labels in legend
    # (see https://stackoverflow.com/a/46160465)
    handles, labels = ax.get_legend_handles_labels()
    order = [2, 0, 1]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])
    
    plt.tight_layout()
    fig.savefig(outpath + '.svg')
    
    # also create html file to go with this
    # Needs to look something like:
    # "<a href="html/US.html"><img src='{attach}fig-homepage-example-plot.svg'></a>"
    # fig.savefig(outpath + '.png', dpi=250)
    with open("pelican/content/pages/index-included-for-figure1-html", "tw") as f:
        f.write(f'<a href="html/{country}.html">')
        f.write('<img src="{attach}fig-homepage-example-plot.svg"></a>')

# save_frontpage_figure('United Kingdom')

In [ ]:
def find_countries_with_highest_cases_deaths(n=5):
    d, c = corona.fetch_deaths(), corona.fetch_cases()
    # most deaths:
    d2 = d.iloc[:, -1]    # last column is most recent date
    c2 = c.iloc[:, -1]
    top_n_deaths = d2.sort_values(ascending=False).head(n)
    top_n_cases = c2.sort_values(ascending=False).head(n)
    
    return top_n_cases, top_n_deaths

def pick_random_country(n=5):
    # use infections
    cases, deaths = find_countries_with_highest_cases_deaths(n=n)
    return random.choice(cases.index)
    

In [ ]:
print(list(find_countries_with_highest_cases_deaths(n=10)[0].index))

todays_country = pick_random_country(n=10)
print(f"Chosen country is {todays_country}")
save_frontpage_figure(todays_country)

In [ ]:
save_frontpage_figure("United Kingdom")